In [59]:
import keras
import tensorflow as tf
import numpy
import pandas
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

In [61]:
dataset = pandas.read_csv("Dane1.csv", index_col=False)
dataset = dataset.iloc[:, 1:]
train, test = train_test_split(dataset, test_size=0.2)
dataset.head()

,target,id,date,flag,username,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [65]:

ds_train = tf.data.Dataset.from_tensor_slices((train["tweet"], train["target"]))
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(32)
print(ds_train.element_spec)

ds_test = tf.data.Dataset.from_tensor_slices((test["tweet"], test["target"]))
ds_test = ds_test.batch(32)
ds_test = ds_test.prefetch(32)
print(ds_test.element_spec)


(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))
(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))


In [57]:
from utils import prepare_text

VOCAB_SIZE = 1000
encoder = keras.layers.TextVectorization(
    standardize = prepare_text, 
    max_tokens=VOCAB_SIZE)
encoder.adapt(ds.map(lambda text, label: text))
encoder.get_vocabulary()


['',
 '[UNK]',
 'i',
 'to',
 'the',
 'a',
 'my',
 'and',
 'is',
 'you',
 'in',
 'for',
 'it',
 'of',
 'on',
 'have',
 'so',
 'but',
 'that',
 'just',
 "i'm",
 'me',
 'at',
 'with',
 'be',
 'not',
 'was',
 'this',
 'good',
 'get',
 'up',
 '-',
 'are',
 'out',
 'like',
 'go',
 "it's",
 'all',
 'your',
 'day',
 'no',
 'now',
 'going',
 'got',
 'do',
 'work',
 'love',
 'from',
 'am',
 'what',
 'back',
 'about',
 'will',
 "don't",
 'we',
 'some',
 'had',
 'can',
 'one',
 'too',
 'new',
 'really',
 'time',
 'im',
 'still',
 'if',
 "can't",
 'its',
 'how',
 'today',
 'u',
 'see',
 'know',
 'want',
 'as',
 'more',
 'when',
 'think',
 'last',
 'an',
 'has',
 'need',
 'they',
 'hope',
 '&amp;',
 'lol',
 'been',
 'off',
 '2',
 'morning',
 'then',
 'or',
 'only',
 'much',
 'there',
 'miss',
 'thanks',
 'very',
 'oh',
 'wish',
 'twitter',
 'feel',
 'home',
 'well',
 'would',
 'he',
 'getting',
 'by',
 'her',
 'great',
 'why',
 'here',
 'make',
 "i'll",
 'night',
 'did',
 'happy',
 'bad',
 'should',

In [58]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])


In [68]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [69]:
history = model.fit(ds_train, epochs=10,
                    validation_data=ds_test,
                    validation_steps=30)


Epoch 1/10
1000/1000 [==============================] - 26s 21ms/step - loss: -77.4678 - accuracy: 0.0442 - val_loss: -179.0695 - val_accuracy: 0.0000e+00
Epoch 2/10
   1/1000 [..............................] - ETA: 18s - loss: -116.0877 - accuracy: 0.0000e+00

KeyboardInterrupt: 